In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from scipy import stats
from geopy.geocoders import Nominatim 

In [2]:
#Read csv file into dataframe 
df = pd.read_csv('data_combined.csv')
df

,Unnamed: 0,Date / Time,City,State,Country,Shape,Duration,Summary,Posted,Images
0,0,1/31/20 21:30,Prospect,OR,USA,Light,30 minute,1.31.2020 Approximately 9:30-10 While driving ...,2/7/20,NaN
1,1,1/31/20 19:57,Arizona City,AZ,USA,Unknown,2:55,Blinking lights tha pull away and tilt.,5/1/20,NaN
2,2,1/31/20 19:56,Arizona City,AZ,USA,Unknown,2,Blinking unexplained lights that tilt and move.,2/25/20,NaN
3,3,1/31/20 19:30,Sevierville,TN,USA,Oval,3-5 minutes,This is twice in one week I've seen this. Roun...,2/7/20,NaN
4,4,1/31/20 18:45,Cornelius,OR,USA,NaN,NaN,5 bright orange lights traveling in drunken ma...,2/7/20,NaN
...,...,...,...,...,...,...,...,...,...,...
14441,14441,10/1/22 15:34,Huntsville,AL,USA,Cigar,2 - 3 minutes,Cigar shaped with a bright circular light to t...,10/8/22,Yes
14442,14442,10/1/22 11:50,Gulf breeze,FL,USA,Circle,Seconds intermittent,This was circular and was blur...I've never se...,10/8/22,NaN
14443,14443,10/1/22 09:52,Redwood valley,CA,USA,Other,NaN,A very very large huge X flying in and out of ...,10/8/22,NaN
14444,14444,10/1/22 04:30,Bountiful,UT,USA,Light,45 seconds,deep above maybe outer space,10/8/22,NaN


DATA CLEANING 

In [3]:
#drop the column 
to_drop = ["Unnamed: 0",'Images','Posted','Summary']
df.drop(to_drop, inplace = True, axis = 1)
df.head()

,Date / Time,City,State,Country,Shape,Duration
0,1/31/20 21:30,Prospect,OR,USA,Light,30 minute
1,1/31/20 19:57,Arizona City,AZ,USA,Unknown,2:55
2,1/31/20 19:56,Arizona City,AZ,USA,Unknown,2
3,1/31/20 19:30,Sevierville,TN,USA,Oval,3-5 minutes
4,1/31/20 18:45,Cornelius,OR,USA,NaN,NaN


In [4]:
df = df[['City', 'State', 'Country']]


In [13]:
#NA in each column
#df['City'].isna().sum() #125 -> 100v -> 64
#df['State'].isna().sum() #576 -> 483 -> 447
#df['Country'].isna().sum() #99 -> 0 

0

In [5]:
#remove NAs from Country column 
df = df.dropna(subset = ['Country'])

In [9]:
df = df.dropna(subset = ['City','State'], how = 'all')

CONVERT LOCATION TO LONG AND LAT

In [14]:
df.loc[df['State'].isnull(),'location'] = df.City + "," + df.Country

C:\Users\jonas\AppData\Local\Temp\ipykernel_11588\1351861423.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['State'].isnull(),'location'] = df.City + "," + df.Country


In [17]:
df.loc[df['City'].isnull(),'location'] = df.State + "," + df.Country

C:\Users\jonas\AppData\Local\Temp\ipykernel_11588\4232419874.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['City'].isnull(),'location'] = df.State + "," + df.Country


In [20]:
loc = Nominatim(user_agent = "GetLoc")
df['gcode'] = df.location.apply(loc.geocode)

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Northeast+of+Mar+Vista%2C+Los+Angeles%2CCA%2CUSA&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [21]:
df

,City,State,Country,location
0,Prospect,OR,USA,"Prospect,OR,USA"
1,Arizona City,AZ,USA,"Arizona City,AZ,USA"
2,Arizona City,AZ,USA,"Arizona City,AZ,USA"
3,Sevierville,TN,USA,"Sevierville,TN,USA"
4,Cornelius,OR,USA,"Cornelius,OR,USA"
...,...,...,...,...
14441,Huntsville,AL,USA,"Huntsville,AL,USA"
14442,Gulf breeze,FL,USA,"Gulf breeze,FL,USA"
14443,Redwood valley,CA,USA,"Redwood valley,CA,USA"
14444,Bountiful,UT,USA,"Bountiful,UT,USA"


In [18]:
df.to_csv('check_na.csv')